In [1]:
from TimewiseKfold import *
from Metrics import *
import numpy as np 
import pandas as pd 
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [3]:
folds, PROVINCES_NUM = splitting_data(region=2)

In [6]:
# Number of trees in random forest
n_estimators = [100, 200, 300, 1000]
# Number of features to consider at every split
max_features = [2, 3]
# Maximum number of levels in tree
max_depth = [80, 90, 100, 110]
# Minimum number of samples required to split a node
min_samples_split = [8, 10, 12]
# Minimum number of samples required at each leaf node
min_samples_leaf = [3, 4, 5]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [7]:
table = metrics_table()

tscv = TimeSeriesSplit(n_splits=2, test_size=PROVINCES_NUM)

for (i, fold) in enumerate(folds):
    X_train, X_test, y_train, y_test = split_train_test(fold=fold)
    
    rf = RandomForestRegressor()
    
    rf_grid = GridSearchCV(estimator=rf, param_grid=random_grid, cv=tscv, n_jobs=-1)
    
    rf_grid.fit(X_train, y_train)
    
    row = [MAXIMUM_YEAR - FOLD_NUM + 1 + i]
    
    row += metrics_calculation(y_actual=y_test, y_pred=rf_grid.predict(X_test), y_train=y_train)
    
    table.loc[len(table)] = row
    
    del rf
    del rf_grid

table  

c:\Application\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Application\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Application\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,Year,RMSE,R2_Score,MSE,MAE,RAE,RSE
0,2018.0,1.553696,0.182645,2.413973,1.240163,0.865068,0.889273
1,2019.0,1.229232,0.049853,1.511011,1.090932,1.056090,0.955169
2,2020.0,5.989578,0.000588,35.875046,3.570698,0.961329,1.400992
